# Talks markdown generator for academicpages

Takes a TSV of talks with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `talks.py`. Run either from the `markdown_generator` folder after replacing `talks.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases, rather than Stuart's non-standard TSV format and citation style.

In [1]:
import pandas as pd
import os

## Data format

The TSV needs to have the following columns: title, type, url_slug, venue, date, location, talk_url, description, with a header at the top. Many of these fields can be blank, but the columns must be in the TSV.

- Fields that cannot be blank: `title`, `url_slug`, `date`. All else can be blank. `type` defaults to "Talk" 
- `date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. 
    - The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/talks/YYYY-MM-DD-[url_slug]`
    - The combination of `url_slug` and `date` must be unique, as it will be the basis for your filenames

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [2]:
!cat ../_talks/YllaLab_Talks.tsv

title	type	url_slug	venue	date	location	talk_url	description
The new FBBB’s Laboratory of Bioinformatics and Genome Biology: computational approaches to study genes, genomes, gene regulatory networks	Talk	jubilee	The Jubilee Conference of the Faculty of Biochemistry and Biotechnology at the Jagiellonian University.	24/09/2021	Jagiellonian University, Krakow, Poland	https://jubileuszwbbib.confer.uj.edu.pl/en_GB/program/ksiazka-streszczen	[Photo1](JW210926_356.jpg)[Photo2](JW210926_360.jpg)[Photo3](JW210926_363.jpg)
Small RNAs: when the small rule the bigger	Talk	winterschool22	XLIX Winter School of the Faculty of Biochemistry and Biotechnology at the Jagiellonian University.	22/02/2022	Jagiellonian University, Krakow, Poland	https://winterschool.wbbib.uj.edu.pl/en_GB/	Guillem Ylla gave a talk in the workshop titled “Omics research – from methodology to application” about the role of small RNAs controlling the expression of mRNAs. ![](WBBiB_Winter_School_2022_slide2.jpg)
Could E93 play t

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [3]:
talks = pd.read_csv("../_talks/YllaLab_Talks.tsv", sep="\t", header=0)
talks

,title,type,url_slug,venue,date,location,talk_url,description
0,The new FBBB’s Laboratory of Bioinformatics an...,Talk,jubilee,The Jubilee Conference of the Faculty of Bioch...,24/09/2021,"Jagiellonian University, Krakow, Poland",https://jubileuszwbbib.confer.uj.edu.pl/en_GB/...,[Photo1](JW210926_356.jpg)[Photo2](JW210926_36...
1,Small RNAs: when the small rule the bigger,Talk,winterschool22,XLIX Winter School of the Faculty of Biochemis...,22/02/2022,"Jagiellonian University, Krakow, Poland",https://winterschool.wbbib.uj.edu.pl/en_GB/,Guillem Ylla gave a talk in the workshop title...
2,Could E93 play the adult specifier role in the...,Talk,ice2022,International Congress of Entomology 2022.,20/07/2022,"Helsinki, Finland",https://ice2020helsinki.fi/,NaN
3,Comparative genomics of insects to unveil the ...,Talk,Bibtorun2022,Bioinformatics in Torun conference by the Poli...,24/06/2022,"Torun, Poland",https://www.ptbi.org.pl/website/conferences/bi...,NaN
4,Cricket genomes: unveiling the genomes of futu...,Talk,PCG2022,6th Polish Congress of Genetics,30/06/2022,"Krakow, Pland",https://kongresgenetyki2022.jordan.pl/,[Photo4](IMG_0161.jpg)
5,Non-traditional insect models are key for unde...,Talk,NtrAMS,Non-Traditional Arthropod Model Systems Meetin...,03/08/2023,"Bowdoin College, Brunswick, Maine, USA",https://www.bowdoin.edu/events/conferences/ntr...,NaN
6,Using bioinformatics to unveil the mysteries o...,Talk,ICG-18,18th Annual Meeting of the International Confe...,26/07/2023,"Shenzhen, China, July 26h 2023",https://icg.bio/en/ICG18index/index.aspx,NaN
7,Using bioinformatics to unveil the mysteries o...,Talk,Genomica,7th Polish Conference of Genetics: “Genomica”,20/05/2023,"Jagiellonian University, Krakow, Poland",http://www.genomica.pl/en/home-page/,NaN


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [4]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    if type(text) is str:
        return "".join(html_escape_table.get(c,c) for c in text)
    else:
        return "False"

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [5]:
loc_dict = {}

for row, item in talks.iterrows():
    
    md_filename = str(item.date) + "-" + item.url_slug + ".md"
    html_filename = str(item.date) + "-" + item.url_slug 
    year = item.date[:4]
    
    md = "---\ntitle: \""   + item.title + '"\n'
    md += "collection: talks" + "\n"
    
    if len(str(item.type)) > 3:
        md += 'type: "' + item.type + '"\n'
    else:
        md += 'type: "Talk"\n'
    
    md += "permalink: /talks/" + html_filename + "\n"
    
    if len(str(item.venue)) > 3:
        md += 'venue: "' + item.venue + '"\n'
        
    if len(str(item.location)) > 3:
        md += "date: " + str(item.date) + "\n"
    
    if len(str(item.location)) > 3:
        md += 'location: "' + str(item.location) + '"\n'
           
    md += "---\n"
    
    
    if len(str(item.talk_url)) > 3:
        md += "\n[More information here](" + item.talk_url + ")\n" 
        
    
    if len(str(item.description)) > 3:
        md += "\n" + html_escape(item.description) + "\n"
        
        
    md_filename = os.path.basename(md_filename)
    #print(md)
    
    with open("../_talks/" + md_filename, 'w') as f:
        f.write(md)

These files are in the talks directory, one directory below where we're working from.

In [6]:
!ls ../_talks

2021-jubilee.md		2023-Genomica.md  JW210926_360.jpg
2022-Bibtorun2022.md	2023-ICG-18.md	  JW210926_363.jpg
2022-ice2022.md		2023-NtrAMS.md	  WBBiB_Winter_School_2022_slide2.jpg
2022-PCG2022.md		IMG_0161.jpg	  YllaLab_Talks.tsv
2022-winterschool22.md	JW210926_356.jpg


In [7]:
!cat ../_talks/2013-03-01-tutorial-1.md

cat: ../_talks/2013-03-01-tutorial-1.md: No such file or directory
